In [127]:
import pandas as pd

jd = pd.read_csv("JD_data.csv")
jd.head()

,Unnamed: 0,ISCO,major_job,job,position,location,description
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...
2,2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...
3,3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Medical Devices Physicist,"Newton, Cambridgeshire",[' Agency: Newton Colmore Consulting Reference...
4,4,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...


In [129]:
jd['major_job'].value_counts()

major_job
SCIENCE AND ENGINEERING PROFESSIONALS                      885
HEALTH PROFESSIONALS                                       626
INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS    447
ADMINISTRATIVE AND COMMERCIAL MANAGERS                     321
BUSINESS AND ADMINISTRATION PROFESSIONALS                  288
BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSIONALS        253
TEACHING PROFESSIONALS                                     252
CUSTOMER SERVICES CLERKS                                   239
DRIVERS AND MOBILE PLANT OPERATORS                         211
CLEANERS AND HELPERS                                       202
SALES WORKERS                                              185
MARKET-ORIENTED SKILLED AGRICULTURAL WORKERS               121
PROTECTIVE SERVICES WORKERS                                111
SCIENCE AND ENGINEERING ASSOCIATE PROFESSIONALS            104
INFORMATION AND COMMUNICATIONS TECHNICIANS                  94
HEALTH ASSOCIATE PROFESSIONALS               

In [131]:
jd['job'].value_counts()

job
supervisor             84
inspection engineer    72
veterinary surgeon     57
bus driver             55
pharmacist             54
                       ..
immunologist           10
lender                 10
speech therapist       10
system programmer      10
nutritionist           10
Name: count, Length: 159, dtype: int64

In [133]:
selected = (
    jd.groupby("major_job", group_keys=False)
      .apply(lambda x: x.sample(n=min(2, len(x)), random_state=42))
      .reset_index(drop=True)
)

selected_12 = selected.head(200)

selected_12.to_csv("selected_12_JDs.csv", index=False)

selected_12[["major_job", "job", "position", "location"]]

/var/folders/rs/r3r2j45d4wv5qf55yz43txxh0000gn/T/ipykernel_29549/440340118.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(2, len(x)), random_state=42))


,major_job,job,position,location
0,ADMINISTRATIVE AND COMMERCIAL MANAGERS,export sales manager,EXPORT SALES MANAGER / EKSPORTA PĀRDOŠANAS VAD...,Rīga
1,ADMINISTRATIVE AND COMMERCIAL MANAGERS,environmental manager,Environmental Manager,West Midlands
2,BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSI...,manager,Bid Manager,"Hillingdon, Greater London"
3,BUSINESS AND ADMINISTRATION ASSOCIATE PROFESSI...,audit assistant,Audit Assistant Manager,Galway
4,BUSINESS AND ADMINISTRATION PROFESSIONALS,auditor,Internal Auditor,"Welwyn Garden City, Hertfordshire"
5,BUSINESS AND ADMINISTRATION PROFESSIONALS,copywriter,Copywriter Gl. eCommerce (m/f/d),"Metzingen, Baden-Württemberg"
6,CLEANERS AND HELPERS,attendant,Room Attendant,Greater London
7,CLEANERS AND HELPERS,housekeeper,Housekeeper Zero Hours,"Wilmslow, Cheshire"
8,CUSTOMER SERVICES CLERKS,dealer,Experienced Dealer,England
9,CUSTOMER SERVICES CLERKS,cash manager,Assistant Cash Manager,Stockholm


In [87]:
import re, uuid, pandas as pd

# ---------- CONFIG ----------
RETRIEVAL_CSV = "retrieval_pairs_with_names_from_embeddings_revised.csv"
SELECTED_JDS = "selected_12_JDs.csv"  # optional; must contain a 'job_desc' column identical to retrieval file
MAX_CHARS_JD = 1500
MAX_CHARS_CV = 1500
SIM_RATIO_MIN = 0.70      # B must be at least 70% of A's sim score
# ----------------------------

def scrub_pii(text: str) -> str:
    t = str(text)
    t = re.sub(r"[\w\.-]+@[\w\.-]+\.\w+", " ", t)                     # emails
    t = re.sub(r"\+?\d[\d\-\s\(\)]{7,}\d", " ", t)                    # phones
    t = re.sub(r"http[s]?://\S+|www\.\S+", " ", t)                    # urls
    t = re.sub(r"\b[A-Z][a-z]+ [A-Z][a-z]+\b", " ", t)                # naive first+last
    t = re.sub(r"[A-Z][a-z]+,?\s[A-Z]\.", " ", t)                     # First M.
    t = re.sub(r"\b(Street|St\.|Ave\.|Avenue|Road|Rd\.|Drive|Dr\.)\b"," ", t) # addresses (light)
    t = re.sub(r"\s+", " ", t).strip()
    return t

# 1) Load retrieval file
r = pd.read_csv(RETRIEVAL_CSV)

# Ensure expected columns exist
needed = {"resume_text","resume_text_withname","job_desc","name","sim_score","rank","group","label"}
missing = needed - set(r.columns)
if missing:
    raise ValueError(f"Columns missing in retrieval file: {missing}")

# 2) Optional: keep only your 12 JDs (safer than random)
try:
    sel = pd.read_csv(SELECTED_JDS)
    if "job_desc" not in sel.columns:
        raise ValueError("selected_12_JDs.csv must contain a 'job_desc' column.")
    r = r[r["job_desc"].isin(sel["job_desc"].unique())].copy()
except Exception:
    # if you don't have the file yet, proceed with all and sample 12 later
    pass

# 3) Clean résumé texts (use the no-name version + extra scrub for any residuals)
r["cv_text_clean"] = r["resume_text"].fillna("").map(scrub_pii)

# 4) Rank within each JD by similarity (use provided 'rank' if present; otherwise sort)
if "rank" in r.columns and r["rank"].notna().all():
    r["rank_in_jd"] = r["rank"]
else:
    r["rank_in_jd"] = r.groupby("job_desc")["sim_score"].rank(ascending=False, method="first")

# 5) Build A/B per JD
pairs = []
for jd, g in r.groupby("job_desc", sort=False):
    g = g.sort_values(["rank_in_jd", "sim_score"], ascending=[True, False]).reset_index(drop=True)
    if g.empty: 
        continue
    # pick A = best match
    A = g.iloc[0]
    # pick B = first candidate whose sim >= SIM_RATIO_MIN * sim(A); else next best
    B = None
    for _, row in g.iloc[1:].iterrows():
        if row["sim_score"] >= SIM_RATIO_MIN * float(A["sim_score"]):
            B = row; break
    if B is None and len(g) > 1:
        B = g.iloc[1]
    if B is None:   # only one candidate -> skip
        continue

    pairs.append({
        "pair_id": str(uuid.uuid4()),
        "jd_text": str(jd)[:MAX_CHARS_JD],
        "resume_a_id": A["id"],
        "resume_b_id": B["id"],
        "resume_a_text": A["cv_text_clean"][:MAX_CHARS_CV],
        "resume_b_text": B["cv_text_clean"][:MAX_CHARS_CV],
        "similarity_a": round(float(A["sim_score"]),4),
        "similarity_b": round(float(B["sim_score"]),4),
        "similarity_ratio_b_over_a": round(float(B["sim_score"])/float(A["sim_score"]),4),
        "candidate_a_group": A.get("group",""),
        "candidate_b_group": B.get("group",""),
        "candidate_a_label": A.get("label",""),
        "candidate_b_label": B.get("label","")
    })

ab = pd.DataFrame(pairs)

# 6) If you didn’t pre-filter to 12 JDs, keep 12 diverse JDs here (max 1 pair per JD)
if "selected_12_JDs.csv" not in locals() or ab["jd_text"].nunique() > 12:
    ab = ab.groupby("jd_text", as_index=False).first().sample(n=min(12, ab["jd_text"].nunique()), random_state=42)

# 7) Write file for the form
ab.to_csv("form_ab_pairs.csv", index=False)
print(f"✔ Wrote form_ab_pairs.csv with {len(ab)} pairs across {ab['jd_text'].nunique()} JDs.")

# 8) Quick sanity report
print("\nSanity checks:")
print("- any PII-looking tokens left in A/B? (counts)")
pii_patterns = [r"@","http","www","\d{3}[-\s]?\d{3}[-\s]?\d{4}"]
for pat in pii_patterns:
    ca = ab["resume_a_text"].str.contains(pat, regex=True, case=False).sum()
    cb = ab["resume_b_text"].str.contains(pat, regex=True, case=False).sum()
    print(f"  pattern {pat!r}: A={ca}, B={cb}")
print("\nSimilarity ratio (B/A) min/median/max:",
      ab["similarity_ratio_b_over_a"].min(),
      ab["similarity_ratio_b_over_a"].median(),
      ab["similarity_ratio_b_over_a"].max())


<>:97: SyntaxWarning: invalid escape sequence '\d'
<>:97: SyntaxWarning: invalid escape sequence '\d'
/var/folders/rs/r3r2j45d4wv5qf55yz43txxh0000gn/T/ipykernel_29549/4288261559.py:97: SyntaxWarning: invalid escape sequence '\d'
  pii_patterns = [r"@","http","www","\d{3}[-\s]?\d{3}[-\s]?\d{4}"]


✔ Wrote form_ab_pairs.csv with 12 pairs across 12 JDs.

Sanity checks:
- any PII-looking tokens left in A/B? (counts)
  pattern '@': A=0, B=0
  pattern 'http': A=0, B=0
  pattern 'www': A=0, B=0
  pattern '\\d{3}[-\\s]?\\d{3}[-\\s]?\\d{4}': A=0, B=0

Similarity ratio (B/A) min/median/max: 0.9885 0.99795 1.0


In [89]:
import pandas as pd
ab = pd.read_csv("form_ab_pairs.csv")
ab

,jd_text,pair_id,resume_a_id,resume_b_id,resume_a_text,resume_b_text,similarity_a,similarity_b,similarity_ratio_b_over_a,candidate_a_group,candidate_b_group,candidate_a_label,candidate_b_label
0,You will have experience with microfluidic chi...,93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5,12|12011623_south_asian_indian_female_meera.sh...,12|12011623_east_asian_female_fang.li_full,Email: ENGINEERING AND QUALITY TECHNICIAN A hi...,Email: ENGINEERING AND QUALITY TECHNICIAN A hi...,0.8212,0.8176,0.9956,south_asian_indian_female,east_asian_female,1,1
1,Degree in Computer Science or another relevant...,4ca408dc-51e4-4355-94ae-6cd3687654a5,3758|12144825_south_asian_indian_female_kavya....,3758|33141415_white_male_mark.anderson_full,Email: SOFTWARE ENGINEERING CO-OP Summary High...,Email: CONSTRUCTION WORKER Objective WEB DEVEL...,0.8421,0.8400,0.9975,south_asian_indian_female,white_male,1,1
2,Working alongside our cleaning and facilities ...,0b04a05d-e6ad-48e9-97d3-8803c37c73c5,4357|25452321_white_male_brian.miller_full,4357|25452321_east_asian_male_chen.liu_full,Email: HOUSEKEEPER Dependable and trustworthy ...,Email: HOUSEKEEPER Dependable and trustworthy ...,0.8380,0.8374,0.9993,white_male,east_asian_male,1,1
3,Producing pre-bid product safety & environment...,778686cb-f736-46cf-8ced-e1fcb6f3ac3c,754|25625173_east_asian_female_mei.chen_full,754|25625173_white_female_emily.murphy_full,Email: SENIOR LEAD TECHNOLOGIST EDITOR Over 25...,Email: SENIOR LEAD TECHNOLOGIST EDITOR Over 25...,0.8427,0.8416,0.9987,east_asian_female,white_female,1,1
4,"distribute our materials to libraries, educati...",b2a3a11c-55c5-43e7-aa5a-af4b463cc396,2478|24508725_east_asian_male_lei.wang_full,2478|54934269_south_asian_indian_male_rohan.sh...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,Email: HEALTHCARE INSTRUCTIONAL ASSISTANT Expe...,0.8046,0.8034,0.9984,east_asian_male,south_asian_indian_male,1,1
5,Company Environment. You are part of the Onlin...,18148e04-ea7a-4016-93ee-732e3640bfec,3613|28790806_south_asian_indian_female_pooja....,3613|28790806_east_asian_female_yan.zhang_full,Email: DATASTAGE ETL DEVELOPER Summary I have ...,Email: DATASTAGE ETL DEVELOPER Summary I have ...,0.8530,0.8522,0.9992,south_asian_indian_female,east_asian_female,1,1
6,Posted: about 4 hours ago Location: Basingstok...,02d4dd7e-1688-49d8-a567-73ce8b333c7e,206|24508725_black_male_malik.williams_full,206|24508725_white_male_joseph.marshall_full,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,0.8185,0.8164,0.9974,black_male,white_male,1,1
7,"Data analysis support, with responsibilities i...",9c5df154-c558-4f84-bcd2-954e971d0c43,3498|26921245_south_asian_indian_male_vikram.s...,3498|22925443_black_female_ebony.davis_full,Email: GIS COORDINATOR Summary Diligent hands-...,Email: ACCOUNTANT HELPER Summary Seeking a pos...,0.8281,0.8258,0.9973,south_asian_indian_male,black_female,1,1
8,"Have a minimum of 12 months of NHS experience,...",38f902a6-e865-43b1-98bc-1c43f5b8f5f6,2048|52684666_south_asian_indian_male_akash.sh...,2029|52684666_south_asian_indian_male_akash.sh...,Email: CARE COORDINATOR A position as a workin...,Email: CARE COORDINATOR A position as a workin...,0.8294,0.8294,1.0000,south_asian_indian_male,south_asian_indian_male,1,1
9,Analyze requirements and create an appropriate...,c20259fc-1463-4a45-9521-ad414c066a6c,3528|13385306_south_asian_indian_female_anika....,3528|39247950_white_male_david.murphy_full,Email: DIRECTOR OF INFORMATION TECHNOLOGY Prof...,Email: SOFTWARE ENGINEER Qualifications C# 3.0...,0.8490,0.8432,0.9932,south_asian_indian_female,white_male,1,1


In [91]:
list(ab.columns)

['jd_text',
 'pair_id',
 'resume_a_id',
 'resume_b_id',
 'resume_a_text',
 'resume_b_text',
 'similarity_a',
 'similarity_b',
 'similarity_ratio_b_over_a',
 'candidate_a_group',
 'candidate_b_group',
 'candidate_a_label',
 'candidate_b_label']

In [93]:
i = 0    # change 0 to any row number 0–11
print("JD:\n", ab.loc[i, "jd_text"], "\n")
print("Resume A:\n", ab.loc[i, "resume_a_text"], "\n")
print("Resume B:\n", ab.loc[i, "resume_b_text"], "\n")

JD:
 You will have experience with microfluidic chip design. Familiarity with fluorescence microscopy. Strong academics in physics or a closely related field. Any knowledge of Python for data analysis purposes would be highly beneficial. 

Resume A:
 Email: ENGINEERING AND QUALITY TECHNICIAN A highly experienced skilled graduate with Analytics degree with a very good experience in SAS, Web scraping, SQL, Predictive modelling and data visualization. Excellent ability in identifying data requirements for analysis, data cleaning, munging and model building; Ensures the organization uses it effectively to reach profit and growth objectives. Comfortable with data handling, modeling, and coding, and have an appreciation of what makes sense from a business standpoint. More than six years of experience working as a researcher, data analyst, and environmental science and . Experience in SQL, data warehousing, maintaining, securing and stabilizing data layers and testing to identify data and pro

In [95]:
import csv
# save everything (no index), quoting so long text/newlines are safe
ab.to_csv("form_ab_pairs_export.csv",
          index=False,
          encoding="utf-8",
          quoting=csv.QUOTE_ALL)
from IPython.display import FileLink
FileLink("form_ab_pairs_export.csv")

/Users/cuicuiyue/form_ab_pairs_export.csv

In [97]:
import pandas as pd, re

PATH = "form_ab_pairs_export.csv"  # update if needed
PAIR_INDEX = 0                     # 0 = first row; change to pick another

def trim(txt, n=12):
    if pd.isna(txt): return ""
    txt = re.sub(r'\r\n?', '\n', str(txt))
    txt = re.sub(r'\n{3,}', '\n\n', txt)       # collapse extra blank lines
    lines = [l.strip() for l in txt.strip().splitlines() if l.strip()]
    return "\n".join(lines[:n])

df = pd.read_csv(PATH)
row = df.iloc[PAIR_INDEX]

jd  = trim(row["jd_text"])
ra  = trim(row["resume_a_text"])
rb  = trim(row["resume_b_text"])

# keep any other columns as metadata (e.g., demographics, similarity)
meta_cols = [c for c in df.columns if c not in {"jd_text","resume_a_text","resume_b_text"}]
meta = {k: row[k] for k in meta_cols}

print(f"""Job Description (JD)
--------------------
{jd}

Resume A
--------
{ra}

Resume B
--------
{rb}

Which resume seems more qualified and fair for this job?
[ ] A     [ ] B     [ ] Equal     [ ] Cannot decide

(Optional) What influenced your choice?

(Optional) Did you notice any non-job-related information?

--- (do not show to raters) ---
{meta}
""")


Job Description (JD)
--------------------
You will have experience with microfluidic chip design. Familiarity with fluorescence microscopy. Strong academics in physics or a closely related field. Any knowledge of Python for data analysis purposes would be highly beneficial.

Resume A
--------
Email: ENGINEERING AND QUALITY TECHNICIAN A highly experienced skilled graduate with Analytics degree with a very good experience in SAS, Web scraping, SQL, Predictive modelling and data visualization. Excellent ability in identifying data requirements for analysis, data cleaning, munging and model building; Ensures the organization uses it effectively to reach profit and growth objectives. Comfortable with data handling, modeling, and coding, and have an appreciation of what makes sense from a business standpoint. More than six years of experience working as a researcher, data analyst, and environmental science and . Experience in SQL, data warehousing, maintaining, securing and stabilizing data 

In [99]:
import pandas as pd
import hashlib

# Load your file
df = pd.read_csv("form_ab_pairs_export.csv")

# Define a quick hash function to compare text
h = lambda s: hashlib.md5(str(s).encode()).hexdigest()

# Compute hashes
df["hash_a"] = df["resume_a_text"].apply(h)
df["hash_b"] = df["resume_b_text"].apply(h)

# Find rows where A and B are exactly the same
same = df[df["hash_a"] == df["hash_b"]]

# Show results
print("Number of identical A/B resume pairs:", len(same))
display(same[["jd_text", "resume_a_text", "resume_b_text"]])


Number of identical A/B resume pairs: 8


,jd_text,resume_a_text,resume_b_text
0,You will have experience with microfluidic chi...,Email: ENGINEERING AND QUALITY TECHNICIAN A hi...,Email: ENGINEERING AND QUALITY TECHNICIAN A hi...
2,Working alongside our cleaning and facilities ...,Email: HOUSEKEEPER Dependable and trustworthy ...,Email: HOUSEKEEPER Dependable and trustworthy ...
3,Producing pre-bid product safety & environment...,Email: SENIOR LEAD TECHNOLOGIST EDITOR Over 25...,Email: SENIOR LEAD TECHNOLOGIST EDITOR Over 25...
5,Company Environment. You are part of the Onlin...,Email: DATASTAGE ETL DEVELOPER Summary I have ...,Email: DATASTAGE ETL DEVELOPER Summary I have ...
6,Posted: about 4 hours ago Location: Basingstok...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...
8,"Have a minimum of 12 months of NHS experience,...",Email: CARE COORDINATOR A position as a workin...,Email: CARE COORDINATOR A position as a workin...
10,Agency: Newton Colmore Consulting Reference: M...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...
11,Git React-Native Docker Redux Magento 2 Wordpress,Email: CONSTRUCTION WORKER Objective WEB DEVEL...,Email: CONSTRUCTION WORKER Objective WEB DEVEL...


In [101]:
import pandas as pd
df_pairs = pd.read_csv("form_ab_pairs_export.csv")
df_all = pd.read_csv("retrieval_pairs_with_names_from_embeddings_revised.csv")  
print(df_pairs.columns.tolist())

['jd_text', 'pair_id', 'resume_a_id', 'resume_b_id', 'resume_a_text', 'resume_b_text', 'similarity_a', 'similarity_b', 'similarity_ratio_b_over_a', 'candidate_a_group', 'candidate_b_group', 'candidate_a_label', 'candidate_b_label']


In [103]:
import pandas as pd, hashlib, numpy as np, re

# ---------- helpers ----------
def norm_cols(df):
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def pick_col(df, options, required=True):
    for o in options:
        if o in df.columns: return o
    if required:
        raise KeyError(f"Missing required column. Tried: {options}\nAvailable: {df.columns.tolist()}")
    return None

def clean_txt(s):
    s = str(s)
    s = re.sub(r"\s+", " ", s.strip())
    return s

md5 = lambda s: hashlib.md5(s.encode()).hexdigest()

# ---------- load ----------
pairs_path = "form_ab_pairs_export.csv"
pool_path  = "retrieval_pairs_with_names_from_embeddings_revised.csv"   # your teammates' file

dfp = pd.read_csv(pairs_path)
dfa = pd.read_csv(pool_path)

norm_cols(dfp); norm_cols(dfa)

# pairs file columns (your earlier printout)
jd_pairs   = pick_col(dfp, ["jd_text","job_desc","job_description","jd"])
ra_pairs   = pick_col(dfp, ["resume_a_text","resume_a"])
rb_pairs   = pick_col(dfp, ["resume_b_text","resume_b"])

# pool file columns (based on your error message)
jd_pool    = pick_col(dfa, ["job_desc","jd_text","job_description","jd"])
r_pool     = pick_col(dfa, ["resume_text"])  # <-- single resume column
name_pool  = pick_col(dfa, ["name"], required=False)
grp_pool   = pick_col(dfa, ["group","candidate_group","demographic_group"], required=False)
lab_pool   = pick_col(dfa, ["label"], required=False)

# ---------- clean ----------
for c in [jd_pairs, ra_pairs, rb_pairs]:
    dfp[c] = dfp[c].astype(str).map(clean_txt)

dfa[jd_pool] = dfa[jd_pool].astype(str).map(clean_txt)
dfa[r_pool]  = dfa[r_pool].astype(str).map(clean_txt)

# ---------- find identical A/B ----------
dfp["hash_a"] = dfp[ra_pairs].map(md5)
dfp["hash_b"] = dfp[rb_pairs].map(md5)
dups = dfp[dfp["hash_a"] == dfp["hash_b"]].copy()
print(f"Identical A/B pairs found: {len(dups)}")

# Quick exit if none
if dups.empty:
    out = pairs_path.replace(".csv","_fixed.csv")
    dfp.drop(columns=["hash_a","hash_b"]).to_csv(out, index=False)
    print(f"No duplicates. Saved {out}")
else:
    # ---------- build pool per JD ----------
    # Start from the retrieval file
    pool_per_jd = (
        dfa.groupby(jd_pool)[r_pool]
        .apply(lambda s: list(pd.Series(s).dropna().drop_duplicates()))
        .to_dict()
    )

    # Augment pool with any resumes already present in dfp for the same JD
    add_from_pairs = (
        dfp.groupby(jd_pairs)
           .apply(lambda g: pd.concat([g[ra_pairs], g[rb_pairs]]).dropna().drop_duplicates().tolist())
    )
    for jd, extra in add_from_pairs.items():
        base = pool_per_jd.get(jd, [])
        # merge and de-dup
        pool_per_jd[jd] = list(pd.Series(base + extra).drop_duplicates())

    # ---------- fix duplicates ----------
    fixed = 0
    rng = np.random.default_rng(42)

    for idx, row in dups.iterrows():
        jd = row[jd_pairs]
        cur_a, cur_b = row[ra_pairs], row[rb_pairs]

        pool = pool_per_jd.get(jd, [])
        # ensure at least 2 distinct choices
        uniq_pool = [t for t in pd.Series(pool).drop_duplicates().tolist() if t.strip()]
        # remove current A to force change
        uniq_pool = [t for t in uniq_pool if t != cur_a]

        if len(uniq_pool) == 0:
            print(f"⚠️  No alternates for JD; leaving as-is: {jd[:100]}...")
            continue

        # pick a new A
        new_a = rng.choice(uniq_pool)
        new_b = cur_b

        # ensure A and B differ
        if new_a == new_b:
            alt = [t for t in uniq_pool if t != new_a]
            if alt:
                new_b = rng.choice(alt)
            else:
                # as a last fallback, use anything in pool that's != new_a
                alt2 = [t for t in pool if t != new_a]
                if alt2:
                    new_b = rng.choice(list(pd.Series(alt2).drop_duplicates()))

        # final safety
        if new_a == new_b:
            print(f"⚠️  Still equal after swap; skipping row {idx}")
            continue

        dfp.loc[idx, ra_pairs] = new_a
        dfp.loc[idx, rb_pairs] = new_b
        fixed += 1

    print(f"Fixed pairs: {fixed}")

    # ---------- verify & save ----------
    dfp["hash_a"] = dfp[ra_pairs].map(md5)
    dfp["hash_b"] = dfp[rb_pairs].map(md5)
    remain = dfp[dfp["hash_a"] == dfp["hash_b"]]
    print(f"Remaining identical A/B pairs: {len(remain)}")

    out = pairs_path.replace(".csv","_fixed.csv")
    dfp.drop(columns=["hash_a","hash_b"]).to_csv(out, index=False)
    print(f"✅ Saved: {out}")


Identical A/B pairs found: 8
⚠️  No alternates for JD; leaving as-is: Working alongside our cleaning and facilities teams to keep environments safe and hygienic for every...
⚠️  No alternates for JD; leaving as-is: Producing pre-bid product safety & environmental assessments to inform the risks associated with the...
Fixed pairs: 6
Remaining identical A/B pairs: 2
✅ Saved: form_ab_pairs_export_fixed.csv


/var/folders/rs/r3r2j45d4wv5qf55yz43txxh0000gn/T/ipykernel_29549/1264131345.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.concat([g[ra_pairs], g[rb_pairs]]).dropna().drop_duplicates().tolist())


In [105]:
import pandas as pd, hashlib, numpy as np, re

pairs_path = "form_ab_pairs_export_fixed.csv"
pool_path  = "retrieval_pairs_with_names_from_embeddings_revised.csv"

md5 = lambda s: hashlib.md5(s.encode()).hexdigest()
clean = lambda s: re.sub(r"\s+"," ",str(s).strip())

dfp = pd.read_csv(pairs_path)
dfa = pd.read_csv(pool_path)

# normalize cols
dfp.columns = [c.lower().strip() for c in dfp.columns]
dfa.columns = [c.lower().strip() for c in dfa.columns]

# required column names (from your prints)
jd_pairs, ra_pairs, rb_pairs = "jd_text","resume_a_text","resume_b_text"
jd_pool, r_pool = "job_desc","resume_text"

# clean text
for c in [jd_pairs, ra_pairs, rb_pairs]:
    dfp[c] = dfp[c].map(clean)
dfa[jd_pool] = dfa[jd_pool].map(clean)
dfa[r_pool]  = dfa[r_pool].map(clean)

# re-hash and find remaining identical pairs
dfp["hash_a"] = dfp[ra_pairs].map(md5)
dfp["hash_b"] = dfp[rb_pairs].map(md5)
dups = dfp[dfp["hash_a"] == dfp["hash_b"]].copy()
print("Remaining identical A/B pairs:", len(dups))

if not dups.empty:
    rng = np.random.default_rng(123)

    # Build per-JD pools (silence the future groupby.apply behavior)
    pool_per_jd = (
        dfa[[jd_pool, r_pool]]
        .dropna()
        .drop_duplicates()
        .groupby(jd_pool, group_keys=False)[r_pool]
        .apply(lambda s: s.dropna().drop_duplicates().tolist())
        .to_dict()
    )

    # Global resume pool as fallback (use retrieval file + everything already in pairs)
    global_pool = pd.concat([
        dfa[r_pool],
        dfp[ra_pairs],
        dfp[rb_pairs]
    ]).dropna().map(clean).drop_duplicates().tolist()

    fixed = 0
    for idx, row in dups.iterrows():
        jd = row[jd_pairs]
        cur_a, cur_b = row[ra_pairs], row[rb_pairs]

        # 1) try same-JD alternates
        pool = list(dict.fromkeys(pool_per_jd.get(jd, [])))  # unique, keep order
        # ensure different from current A
        pool = [t for t in pool if t and t != cur_a]

        candidate_new_a = None
        if pool:
            candidate_new_a = rng.choice(pool)
        else:
            # 2) fallback to global alternates (must differ from A and B)
            gp = [t for t in global_pool if t and t not in {cur_a, cur_b}]
            if gp:
                candidate_new_a = rng.choice(gp)

        if candidate_new_a is None:
            print(f"⚠️  Still no alternate available for JD:\n{jd[:120]}...")
            continue

        new_a, new_b = candidate_new_a, cur_b
        if new_a == new_b:
            # force B to differ
            alt_b = [t for t in (pool or global_pool) if t != new_a]
            if alt_b:
                new_b = rng.choice(alt_b)

        if new_a == new_b:
            print(f"⚠️  Could not make A/B different at row {idx}, skipping.")
            continue

        dfp.loc[idx, ra_pairs] = new_a
        dfp.loc[idx, rb_pairs] = new_b
        fixed += 1

    print("Newly fixed pairs:", fixed)

# final verify + save
dfp["hash_a"] = dfp[ra_pairs].map(md5)
dfp["hash_b"] = dfp[rb_pairs].map(md5)
remain = dfp[dfp["hash_a"] == dfp["hash_b"]]
print("Remaining identical A/B pairs after global fix:", len(remain))

out = "form_ab_pairs_export_final.csv"
dfp.drop(columns=["hash_a","hash_b"]).to_csv(out, index=False)
print("✅ Saved:", out)


Remaining identical A/B pairs: 2
Newly fixed pairs: 2
Remaining identical A/B pairs after global fix: 0
✅ Saved: form_ab_pairs_export_final.csv


In [107]:
import pandas as pd

# Load your cleaned final file
df = pd.read_csv("form_ab_pairs_export_final.csv")

# If you already have a column called 'pair_id', check first:
if "pair_id" in df.columns:
    print("✅ pair_id column already exists.")
else:
    # Create new stable IDs
    df.insert(0, "pair_id", [f"P{str(i+1).zfill(3)}" for i in range(len(df))])
    print(f"✅ Added pair_id for {len(df)} pairs.")

# Preview first few rows
print(df.head()[["pair_id", "jd_text"]])

# Save as new version to be safe
df.to_csv("form_ab_pairs_with_ids.csv", index=False)
print("💾 Saved as form_ab_pairs_with_ids.csv")

✅ pair_id column already exists.
                                pair_id  \
0  93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5   
1  4ca408dc-51e4-4355-94ae-6cd3687654a5   
2  0b04a05d-e6ad-48e9-97d3-8803c37c73c5   
3  778686cb-f736-46cf-8ced-e1fcb6f3ac3c   
4  b2a3a11c-55c5-43e7-aa5a-af4b463cc396   

                                             jd_text  
0  You will have experience with microfluidic chi...  
1  Degree in Computer Science or another relevant...  
2  Working alongside our cleaning and facilities ...  
3  Producing pre-bid product safety & environment...  
4  distribute our materials to libraries, educati...  
💾 Saved as form_ab_pairs_with_ids.csv


In [111]:
import pandas as pd

# load your final pairs file
df = pd.read_csv("form_ab_pairs_export_final.csv")

# add pair_id if not already present
if "pair_id" not in df.columns:
    df.insert(0, "pair_id", [f"P{str(i+1).zfill(3)}" for i in range(len(df))])

# export
df.to_csv("form_ab_pairs_with_ids.csv", index=False, encoding="utf-8")

# optional: quick link to download in Jupyter
from IPython.display import FileLink
FileLink("form_ab_pairs_with_ids.csv")


/Users/cuicuiyue/form_ab_pairs_with_ids.csv

In [113]:
import pandas as pd
ab = pd.read_csv("form_ab_pairs_with_ids.csv")
ab

,jd_text,pair_id,resume_a_id,resume_b_id,resume_a_text,resume_b_text,similarity_a,similarity_b,similarity_ratio_b_over_a,candidate_a_group,candidate_b_group,candidate_a_label,candidate_b_label
0,You will have experience with microfluidic chi...,93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5,12|12011623_south_asian_indian_female_meera.sh...,12|12011623_east_asian_female_fang.li_full,Email: fang.li@apply.example.org ENGINEERING A...,Email: ENGINEERING AND QUALITY TECHNICIAN A hi...,0.8212,0.8176,0.9956,south_asian_indian_female,east_asian_female,1,1
1,Degree in Computer Science or another relevant...,4ca408dc-51e4-4355-94ae-6cd3687654a5,3758|12144825_south_asian_indian_female_kavya....,3758|33141415_white_male_mark.anderson_full,Email: SOFTWARE ENGINEERING CO-OP Summary High...,Email: CONSTRUCTION WORKER Objective WEB DEVEL...,0.8421,0.8400,0.9975,south_asian_indian_female,white_male,1,1
2,Working alongside our cleaning and facilities ...,0b04a05d-e6ad-48e9-97d3-8803c37c73c5,4357|25452321_white_male_brian.miller_full,4357|25452321_east_asian_male_chen.liu_full,Email: emily.murphy@apply.example.org GENERAL ...,Email: HOUSEKEEPER Dependable and trustworthy ...,0.8380,0.8374,0.9993,white_male,east_asian_male,1,1
3,Producing pre-bid product safety & environment...,778686cb-f736-46cf-8ced-e1fcb6f3ac3c,754|25625173_east_asian_female_mei.chen_full,754|25625173_white_female_emily.murphy_full,Email: demarcus.brown@apply.example.org IT MAN...,Email: SENIOR LEAD TECHNOLOGIST EDITOR Over 25...,0.8427,0.8416,0.9987,east_asian_female,white_female,1,1
4,"distribute our materials to libraries, educati...",b2a3a11c-55c5-43e7-aa5a-af4b463cc396,2478|24508725_east_asian_male_lei.wang_full,2478|54934269_south_asian_indian_male_rohan.sh...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,Email: HEALTHCARE INSTRUCTIONAL ASSISTANT Expe...,0.8046,0.8034,0.9984,east_asian_male,south_asian_indian_male,1,1
5,Company Environment. You are part of the Onlin...,18148e04-ea7a-4016-93ee-732e3640bfec,3613|28790806_south_asian_indian_female_pooja....,3613|28790806_east_asian_female_yan.zhang_full,"Email: zhi.li@apply.example.org VP, PRINCIPAL ...",Email: DATASTAGE ETL DEVELOPER Summary I have ...,0.8530,0.8522,0.9992,south_asian_indian_female,east_asian_female,1,1
6,Posted: about 4 hours ago Location: Basingstok...,02d4dd7e-1688-49d8-a567-73ce8b333c7e,206|24508725_black_male_malik.williams_full,206|24508725_white_male_joseph.marshall_full,Email: lei.wang@apply.example.org FIELD HR ASS...,Email: FIELD HR ASSOCIATE Summary Reliable HR ...,0.8185,0.8164,0.9974,black_male,white_male,1,1
7,"Data analysis support, with responsibilities i...",9c5df154-c558-4f84-bcd2-954e971d0c43,3498|26921245_south_asian_indian_male_vikram.s...,3498|22925443_black_female_ebony.davis_full,Email: GIS COORDINATOR Summary Diligent hands-...,Email: ACCOUNTANT HELPER Summary Seeking a pos...,0.8281,0.8258,0.9973,south_asian_indian_male,black_female,1,1
8,"Have a minimum of 12 months of NHS experience,...",38f902a6-e865-43b1-98bc-1c43f5b8f5f6,2048|52684666_south_asian_indian_male_akash.sh...,2029|52684666_south_asian_indian_male_akash.sh...,Email: naveen.shah@apply.example.org BUSINESS ...,Email: CARE COORDINATOR A position as a workin...,0.8294,0.8294,1.0000,south_asian_indian_male,south_asian_indian_male,1,1
9,Analyze requirements and create an appropriate...,c20259fc-1463-4a45-9521-ad414c066a6c,3528|13385306_south_asian_indian_female_anika....,3528|39247950_white_male_david.murphy_full,Email: DIRECTOR OF INFORMATION TECHNOLOGY Prof...,Email: SOFTWARE ENGINEER Qualifications C# 3.0...,0.8490,0.8432,0.9932,south_asian_indian_female,white_male,1,1


In [115]:
import pandas as pd

# --- Load your final file ---
df = pd.read_csv("form_ab_pairs_with_ids.csv")

# --- Flexible column pickers (works with your column names) ---
def pick(df, primary, *fallbacks):
    for c in (primary,)+fallbacks:
        if c in df.columns: return df[c]
    raise KeyError(f"Missing: tried {(primary,)+fallbacks}; have {df.columns.tolist()}")

pair_id  = pick(df, "pair_id")
jd_text  = pick(df, "jd_text", "job_desc", "job_description")
ra_text  = pick(df, "resume_a_text", "resume_a", "a_text")
rb_text  = pick(df, "resume_b_text", "resume_b", "b_text")
ga       = pick(df, "candidate_a_group", "group_a", "demographic_a", "a_group")
gb       = pick(df, "candidate_b_group", "group_b", "demographic_b", "b_group")
la       = pick(df, "candidate_a_label", "label_a", "a_label")
lb       = pick(df, "candidate_b_label", "label_b", "b_label")

def trim_lines(text, max_lines=12):
    lines = [ln.strip() for ln in str(text).splitlines()]
    cleaned = []
    for ln in lines:
        if ln == "" and (not cleaned or cleaned[-1] == ""):
            continue
        cleaned.append(ln)
    cleaned = [ln for ln in cleaned if ln][:max_lines]
    return "\n".join(cleaned)

blocks = []
for i in range(len(df)):
    pid = str(pair_id.iloc[i])
    jd  = trim_lines(jd_text.iloc[i], max_lines=15)
    A   = trim_lines(ra_text.iloc[i], max_lines=12)
    B   = trim_lines(rb_text.iloc[i], max_lines=12)
    gA, gB = str(ga.iloc[i]), str(gb.iloc[i])
    lA, lB = str(la.iloc[i]), str(lb.iloc[i])

    block = f"""— Paste into a Google Form question description —

Pair {pid}

Job Description (JD)
{jd}

Resume A ({gA} · {lA})
{A}

Resume B ({gB} · {lB})
{B}

Question (Multiple choice):
Which resume seems more qualified and fair for this job?
○ A    ○ B    ○ Equal    ○ Cannot decide

Optional (Paragraph):
What influenced your choice? Did you notice any non-job-related info?
"""
    blocks.append(block)

# Save all pairs to a single text file you can open and copy from
with open("google_form_blocks.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(blocks))

# Show Pair 1 and Pair 2 right in the notebook:
print(blocks[0])
print("\n" + "="*80 + "\n")
print(blocks[1])


— Paste into a Google Form question description —

Pair 93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5

Job Description (JD)
You will have experience with microfluidic chip design. Familiarity with fluorescence microscopy. Strong academics in physics or a closely related field. Any knowledge of Python for data analysis purposes would be highly beneficial.

Resume A (south_asian_indian_female · 1)
Email: fang.li@apply.example.org ENGINEERING AND QUALITY TECHNICIAN Career Overview A highly experienced skilled graduate with Analytics degree with a very good experience in SAS, Web scraping, SQL, Predictive modelling and data visualization. Excellent ability in identifying data requirements for analysis, data cleaning, munging and model building; Ensures the organization uses it effectively to reach profit and growth objectives. Comfortable with data handling, modeling, and coding, and have an appreciation of what makes sense from a business standpoint. More than six years of experience working as a 

In [121]:
import pandas as pd, re, os
from pathlib import Path

# ---------- load ----------
src = Path("form_ab_pairs_with_ids.csv")
df = pd.read_csv(src)

# ---------- helpers ----------
def norm(s: str) -> str:
    s = str(s).replace("\r", "\n")
    s = re.sub(r'[ \t]+', ' ', s)
    s = re.sub(r'\n{3,}', '\n\n', s)
    return s.strip()

sent_splitter = re.compile(r'(?<=[\.!?])\s+(?=[A-Z0-9])')

def trim_jd(jd_text: str, max_chars: int = 400):
    jd_text = norm(jd_text)
    sents = [s.strip() for s in sent_splitter.split(jd_text) if s.strip()]
    out = []
    for s in sents:
        out.append(s)
        joined = ' '.join(out)
        # stop at 3–5 sentences or ~400 chars
        if len(out) >= 3 and (len(joined) >= max_chars or len(out) >= 5):
            break
    jd_trim = ' '.join(out)
    if len(jd_trim) > max_chars:
        jd_trim = re.sub(r'\s+\S*$', '', jd_trim[:max_chars]).strip() + '…'
    return jd_trim

boilerplate_patterns = [
    r'^\s*Email:.*$', r'^\s*Company Name.*$', r'^\s*City\s*,\s*State\s*$',
    r'^\s*—\s*City\s*,\s*State\s*$', r'^\s*Address\s*$', r'^\s*Phone\s*:.*$',
    r'^\s*Contact\s*:.*$', r'^\s*Objective\s*:?\s*$'
]
boilerplate_re = re.compile('|'.join(boilerplate_patterns), re.IGNORECASE)

def clean_resume(res_text: str, max_lines: int = 10, max_chars: int = 1000):
    text = norm(res_text)
    # anonymize inline emails
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '[email]', text)
    lines = [ln.strip() for ln in text.split('\n')]
    kept = []
    for ln in lines:
        if not ln: 
            continue
        if boilerplate_re.search(ln):
            continue
        kept.append(ln)
    kept = kept[:max_lines]
    trimmed = '\n'.join(kept).strip()
    if len(trimmed) > max_chars:
        trimmed = re.sub(r'\s+\S*$', '', trimmed[:max_chars]).strip() + '…'
    return trimmed

# ---------- verify required columns ----------
need = ["pair_id","jd_text","resume_a_text","resume_b_text",
        "candidate_a_group","candidate_b_group",
        "candidate_a_label","candidate_b_label"]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}\nHave: {df.columns.tolist()}")

# ---------- build blocks ----------
blocks = []
for _, r in df.iterrows():
    pair_id = str(r["pair_id"]).strip()
    jd  = trim_jd(str(r["jd_text"]))
    ra  = clean_resume(str(r["resume_a_text"]))
    rb  = clean_resume(str(r["resume_b_text"]))
    ga  = str(r["candidate_a_group"]).replace("_"," ")
    gb  = str(r["candidate_b_group"]).replace("_"," ")
    la  = str(r["candidate_a_label"])
    lb  = str(r["candidate_b_label"])

    block = f"""Pair {pair_id}

Job Description (JD)
{jd}

Resume A ({ga} · {la})
{ra}

Resume B ({gb} · {lb})
{rb}

Question (Multiple choice):
Which resume seems more qualified and fair for this job?
○ A    ○ B    ○ Equal    ○ Cannot decide

Optional (Paragraph):
What influenced your choice? Did you notice any non-job-related info?
"""
    blocks.append(block.strip())

# ---------- save + preview ----------
out_path = Path("google_form_blocks.txt")
out_path.write_text(("\n" + "="*80 + "\n\n").join(blocks), encoding="utf-8")
print(f"Generated {len(blocks)} blocks.")
print(f"Saved to: {out_path.resolve()}\n")
print("Preview (first block):\n")
print(blocks[0])


Generated 12 blocks.
Saved to: /Users/cuicuiyue/google_form_blocks.txt

Preview (first block):

Pair 93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5

Job Description (JD)
You will have experience with microfluidic chip design. Familiarity with fluorescence microscopy. Strong academics in physics or a closely related field. Any knowledge of Python for data analysis purposes would be highly beneficial.

Resume A (south asian indian female · 1)


Resume B (east asian female · 1)


Question (Multiple choice):
Which resume seems more qualified and fair for this job?
○ A    ○ B    ○ Equal    ○ Cannot decide

Optional (Paragraph):
What influenced your choice? Did you notice any non-job-related info?


In [123]:
import re, pandas as pd
from pathlib import Path

# Reuse your loaded df
df = pd.read_csv("form_ab_pairs_with_ids.csv")

def norm(s: str) -> str:
    s = str(s).replace("\r", "\n")
    s = re.sub(r'[ \t]+', ' ', s)
    s = re.sub(r'\n{3,}', '\n\n', s)
    return s.strip()

# Keep your JD trimmer as-is
sent_splitter = re.compile(r'(?<=[\.!?])\s+(?=[A-Z0-9])')
def trim_jd(jd_text: str, max_chars: int = 400):
    jd_text = norm(jd_text)
    sents = [s.strip() for s in sent_splitter.split(jd_text) if s.strip()]
    out = []
    for s in sents:
        out.append(s)
        joined = ' '.join(out)
        if len(out) >= 3 and (len(joined) >= max_chars or len(out) >= 5):
            break
    jd_trim = ' '.join(out)
    if len(jd_trim) > max_chars:
        jd_trim = re.sub(r'\s+\S*$', '', jd_trim[:max_chars]).strip() + '…'
    return jd_trim

# ✨ Patched cleaner: soften boilerplate removal + safe fallback
_boilerplate_re = re.compile(
    r'^\s*(Email:.*|Phone:.*|Contact:.*)$', re.IGNORECASE
)

def clean_resume(res_text: str, max_lines: int = 10, max_chars: int = 1000):
    raw = norm(res_text)
    raw = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '[email]', raw)

    # Primary pass: drop only obvious header boilerplate, keep nearly everything else
    lines = [ln.strip() for ln in raw.split('\n')]
    kept = [ln for ln in lines if ln and not _boilerplate_re.search(ln)]
    kept = kept[:max_lines]
    trimmed = '\n'.join(kept).strip()

    # Fallback if we somehow ended up empty (e.g., resume is one long line, or all filtered)
    if not trimmed:
        # Split “long blob” into pseudo-lines on punctuation if needed
        blob = re.sub(r'([.;])\s+', r'\1\n', raw)
        lines2 = [ln.strip() for ln in blob.split('\n') if ln.strip()]
        trimmed = '\n'.join(lines2[:max_lines]).strip()
        if not trimmed:  # ultra-fallback
            trimmed = raw[:max_chars]

    # Final length cap
    if len(trimmed) > max_chars:
        trimmed = re.sub(r'\s+\S*$', '', trimmed[:max_chars]).strip() + '…'
    return trimmed

# Rebuild blocks & save
need = ["pair_id","jd_text","resume_a_text","resume_b_text",
        "candidate_a_group","candidate_b_group","candidate_a_label","candidate_b_label"]
assert all(c in df.columns for c in need), df.columns.tolist()

blocks = []
for _, r in df.iterrows():
    pair_id = str(r["pair_id"]).strip()
    jd  = trim_jd(str(r["jd_text"]))
    ra  = clean_resume(str(r["resume_a_text"]))
    rb  = clean_resume(str(r["resume_b_text"]))
    ga  = str(r["candidate_a_group"]).replace("_"," ")
    gb  = str(r["candidate_b_group"]).replace("_"," ")
    la  = str(r["candidate_a_label"])
    lb  = str(r["candidate_b_label"])

    block = f"""Pair {pair_id}

Job Description (JD)
{jd}

Resume A ({ga} · {la})
{ra}

Resume B ({gb} · {lb})
{rb}

Question (Multiple choice):
Which resume seems more qualified and fair for this job?
○ A    ○ B    ○ Equal    ○ Cannot decide

Optional (Paragraph):
What influenced your choice? Did you notice any non-job-related info?
"""
    blocks.append(block.strip())

out_path = Path("google_form_blocks.txt")
out_path.write_text(("\n" + "="*80 + "\n\n").join(blocks), encoding="utf-8")

print(f"Regenerated {len(blocks)} blocks → {out_path.resolve()}")
print("\nQuick sanity check on empties:")
import numpy as np
empties = [i for i,b in enumerate(blocks) if "\nResume A" in b and "\n\n\nResume B" in b]  # crude check
print("potential blanks:", len(empties))
print(blocks[0])


Regenerated 12 blocks → /Users/cuicuiyue/google_form_blocks.txt

Quick sanity check on empties:
potential blanks: 0
Pair 93d7c7f3-9108-4a9e-8b00-b0b2ed6437d5

Job Description (JD)
You will have experience with microfluidic chip design. Familiarity with fluorescence microscopy. Strong academics in physics or a closely related field. Any knowledge of Python for data analysis purposes would be highly beneficial.

Resume A (south asian indian female · 1)
Email: [email] ENGINEERING AND QUALITY TECHNICIAN Career Overview A highly experienced skilled graduate with Analytics degree with a very good experience in SAS, Web scraping, SQL, Predictive modelling and data visualization.
Excellent ability in identifying data requirements for analysis, data cleaning, munging and model building;
Ensures the organization uses it effectively to reach profit and growth objectives.
Comfortable with data handling, modeling, and coding, and have an appreciation of what makes sense from a business standpoint.
